In [223]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [224]:
df = pd.read_csv('fake_users_data.csv', index_col=0)

In [225]:
df = pd.get_dummies(df, prefix=['age', 'gender', 'type', 'diff', 'comp', 'trans'], columns=['age', 'gender', 'type_route', 'difficulty','accompaniment', 'transport'])

In [226]:
def mapping(x):
    if x == 'Gratis':
        return 1
    elif x == '1-50 Euros':
        return 2
    elif x == '+50 Euros':
        return 3
    else:
        return 9999

df['price'] = df['price'].apply(mapping)

In [227]:
routes_dictionary ={'Arbres monumentals i singulars':0,
                    'Arbres: La volta al món':1,
                    'Arbres: Els indians':2,
                    'Arbres: Un passeig per l’Albereda':3,
                    'Arbres: Un viatge pel Mediterrani':4,
                    'Ruta Carrer Russafa - la Llotja':5,
                    'Contes Blasco Ibañez – Ruta 1':6,
                    'Contes Blasco Ibañez – Ruta 2':7,
                    'Entre Naranjos':8,
                    'Glorieta-Santa Catalina':9,
                    'Jardins del Reial - Glorieta':10,
                    'La barraca':11,
                    'Ruta Cultural Anell Ciclista':12,
                    'Ruta de la Seda':13,
                    "Ruta dels Pecats a l'entorn del Mercat Central":14,
                    'Ruta pel Centre Històric':15,
                    "Ruta recinte enmurallat d'època islàmica":16,
                    'Ruta València amb bicicleta':17,
                    'Sant Agustí – Pont de Fusta':18,
                    'Sant Agustí - Portal Nou':19,
                    'Serrans - Palau de la Generalitat':20,
                    'Torres de Quart - Temple':21,
                    "Trinitat - L'Almodí":22,
                    'Ruta Valencia en la Memòria':23,
                    'No hay ruta':24}

In [228]:
df = df.rename({'route_id':'route'}, axis=1)

In [229]:
df['route_id'] = df['route'].map(routes_dictionary)

In [230]:
X = df.drop(columns=['route_id','route'], axis=1)
y = df[['route_id']]

In [231]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [232]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=3, random_state=42)

from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
estimator = DecisionTreeClassifier(max_depth=3)
ada = AdaBoostClassifier(base_estimator = estimator, n_estimators=200, learning_rate=0.5, random_state=42)

from sklearn.ensemble import GradientBoostingClassifier
gbct = GradientBoostingClassifier(max_depth=3, n_estimators=100, random_state=42)

from xgboost import XGBRFClassifier
xgb = XGBRFClassifier(random_state=42)

from lightgbm import LGBMClassifier
lgb = LGBMClassifier()

from catboost import CatBoostClassifier
cat = CatBoostClassifier(verbose=0)

In [233]:
models = [log, tree, forest, ada, gbct, cat, lgb, xgb]
names = ['Logistic', 'Decision Tree', 'Random Forest', 'AdaBoost', 'Gradient Boosting', 'CatBoost', 'Lightgbm', 'XGBoost']
scores = []
predictions = {}
for idx, model in enumerate(models):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    predictions[names[idx]] = y_pred
    scores.append(accuracy_score(y_pred, y_test))

c:\Users\Paris\anaconda3\envs\desafio\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paris\anaconda3\envs\desafio\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Paris\AppData\Local\Temp\ipykernel_10668\2367934231.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,),

In [234]:
names = ['Logistic', 'Decision Tree', 'Random Forest', 'AdaBoost', 'Gradient Boosting', 'CatBoost', 'Lightgbm', 'XGBoost']
pd.DataFrame(scores, index=names, columns=['Accuracy']).sort_values(by='Accuracy', ascending=False)

,Accuracy
XGBoost,0.704000
Logistic,0.702667
Lightgbm,0.700333
Gradient Boosting,0.699500
AdaBoost,0.699000
CatBoost,0.697667
Random Forest,0.692000
Decision Tree,0.519667


In [235]:
df_feature = pd.DataFrame(cat.get_feature_importance(), index=X.columns, columns=['feature_importance']).sort_values(by='feature_importance', ascending=False)
df_feature

,feature_importance
diff_Alta,28.688936
age_18-35,24.270747
type_Turistica,8.523921
type_Historica,8.488202
type_Patrimonio,6.761074
trans_A Pie,6.635629
diff_Baja,6.567617
type_Literaria,2.664253
time,2.456156
trans_Bicicleta,1.622730
